In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Switch to the project working directory

In [ ]:
!curl ftp://ftp.inrialpes.fr/pub/lear/douze/data/INRIAPerson.tar -o inria.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  2  969M    2 24.8M    0     0  3747k      0  0:04:24  0:00:06  0:04:18 5129k^C


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
import torch
torch.cuda.is_available()

True

In [2]:
!nvidia-smi

Wed Apr 27 20:25:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd /content/drive/MyDrive/CSE569S/lab3/adversarial-yolo
%ls

/content/drive/MyDrive/CSE569S/lab3/adversarial-yolo
 adversarial_examples.ipynb   how_to_debug.md      predictions.jpg
 appliedpatches/              image.py             __pycache__/
 batch_detect.py              inria/               pytorch041_cuda92_colab.sh
 batch_rotate.py              inria.tar            README.md
 cfg/                         layers/              recall.py
 cfg.py                       LICENSE              region_loss.py
 class_only.json              load_data.py         run_docker.sh
 class_shift.json             median_pool.py       runs/
 clean_results.json           models/              saved_patches/
 craft_adv.py                 noise_results.json   scripts/
 darknet.py                   non_printability/    test/
 data/                       'oude versies'/       test_patch2.py
 dataset.py                   partial.py           test_patch3.py
 debug.py                     patch_config.py      test_patch.py
 demo.py                      patches/          

Install packages that you need here

In [4]:
!pip3 install tensorboardX

     |████████████████████████████████| 125 kB 26.3 MB/s 


In [5]:
pip install torch

In [6]:
import tensorboardX

## 2.1 Attack Human-Detection


### (1) Detect without patch
Utilize the apis in detect.py to recognize images of people. Feel free to edit the source code if it’s needed. The result should be plotted after running the code.



In [ ]:
# Your code starts here
# Hint: To directly run the main function in a file, use 
%run file_name.py param1 param2 ...

ERROR:root:File `'file_name.py'` not found.


In [ ]:
%run train_patch.py 'paper_obj'

In [ ]:
%run detect.py 'cfg/yolov2.cfg' 'weights/yolov2.weights' 'persons/person_079.jpg'

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32
    1 max          2 x 2 / 2   608 x 608 x  32   ->   304 x 304 x  32
    2 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64
    3 max          2 x 2 / 2   304 x 304 x  64   ->   152 x 152 x  64
    4 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128
    5 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64
    6 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128
    7 max          2 x 2 / 2   152 x 152 x 128   ->    76 x  76 x 128
    8 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256
    9 conv    128  1 x 1 / 1    76 x  76 x 256   ->    76 x  76 x 128
   10 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256
   11 max          2 x 2 / 2    76 x  76 x 256   ->    38 x  38 x 256
   12 conv    512  3 x 3 / 1    38 x  38 x 256   ->    38 x  38 x 512
   13 conv    256  1 x 

/content/drive/MyDrive/CSE569S/lab3/adversarial-yolo/utils.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_confs = torch.nn.Softmax()(Variable(output[5:5+num_classes].transpose(0,1))).data


torch.Size([1805, 80])
persons/person_079.jpg: Predicted in 0.625070 seconds.
[0]person: 0.995249
[39]bottle: 0.984207
[56]chair: 0.990320
[73]book: 0.978198
[73]book: 0.998481
save plot results to predictions.jpg


### (2) Train patches
Now we will train two patches by ourselves. One of the patches has a starter image as input and the other one is completely generated by our code. To do these, please 


*   Take a look at train_patch.py and edit the relevant code.
*   Write a wrapped-up api for generating these two kinds of patches.
*   Call the api to generate a with-starter-image patch and a without-starter-image patch.

Please make sure to save these images since you need to use them in the next step.

In [ ]:
# starter_img can be an img path or the string 'no'
def patch_training_caller(starter_img):
  %run train_patch.py 'paper_obj' $starter_img

In [ ]:
# Call patch_training_caller to generate a patch with starter image
# Your code starts here
patch_training_caller('saved_patches/patchnew0.jpg')

In [ ]:
# Call patch_training_caller to generate a patch without starter image
# Your code starts here
patch_training_caller('no')

### (3) Detect people with patches
Use the same api in detect.py as section 2.1 (1) to detect people who hold the patch picture you generated in 2.1 (2). The model should not recognize the people in this case. 


In [ ]:
# Your code starts here
# We did this during class time.  By holding our trained patch in front of ourselves, we were able to fool the object detector into thinking we weren't there

## 2.2 Attack Stop Signs-Detection

### (1) Detect without patch
Similar to 2.1 (1), first we need to use apis in detect.py to recognize images of stop signs.

In [ ]:
# Your code starts here
%run detect.py 'cfg/yolov2.cfg' 'weights/yolov2.weights' 'data/stop.jpg'

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32
    1 max          2 x 2 / 2   608 x 608 x  32   ->   304 x 304 x  32
    2 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64
    3 max          2 x 2 / 2   304 x 304 x  64   ->   152 x 152 x  64
    4 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128
    5 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64
    6 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128
    7 max          2 x 2 / 2   152 x 152 x 128   ->    76 x  76 x 128
    8 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256
    9 conv    128  1 x 1 / 1    76 x  76 x 256   ->    76 x  76 x 128
   10 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256
   11 max          2 x 2 / 2    76 x  76 x 256   ->    38 x  38 x 256
   12 conv    512  3 x 3 / 1    38 x  38 x 256   ->    38 x  38 x 512
   13 conv    256  1 x 

/content/drive/MyDrive/CSE569S/lab3/adversarial-yolo/utils.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_confs = torch.nn.Softmax()(Variable(output[5:5+num_classes].transpose(0,1))).data


## (2) Patch stop signs
* Edit the source code involved in the patch-sticking control flow to attach the patch image on all the stop sign images. Images belonging to any other class (i.e. people, horses, etc.) should not be patched.
* Write a wrapped-up api for sticking patches on stop signs.
* Call the api to stick patch on stop sign. You should specify which patch is used in parameter.


In [8]:
# This function does not take in parameters since test_patch itself was modified to take in the images
def patch_sticking_caller():
  # If there's no stop sign in the image, the image should not be patched.
  # Your code starts here
  %run test_patch3.py

In [9]:
patch_sticking_caller()

starting test read
img read!
starting test read
img read!
Setting everything up
PatchTransformer(
  (medianpooler): MedianPool2d()
)
Done
new image
new image
new image
new image
new image
new image
new image
new image


/content/drive/MyDrive/CSE569S/lab3/adversarial-yolo/utils.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_confs = torch.nn.Softmax()(Variable(output[5:5+num_classes].transpose(0,1))).data


torch.Size([1805, 80])
before transform
in patch transform
skipped all noise shit
AAAAAAAAAAAAAAAAAAAAA
after transform


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:4256: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:4194: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


torch.Size([1805, 80])
new image
new image
new image
new image


### (3) Detect stop sign with patch
Use the same api in detect.py as section 2.2 (1) to detect the patched stop sign. The model should not recognize the stop sign in this case. 

In [ ]:
# Your code starts here
# We did this part in class as well.  Using our trained patch we were able to fool the object detection network and make the car run over the stop sign-=